<a href="https://colab.research.google.com/github/Dee-Nwanjah/SQL-Database-Fundamental-Projects/blob/main/7.)Customer_Analytics_System(MAIN_PROJECT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# SETUP AND IMPORTS
# ==============================================================================

# Install required packages
import subprocess
import sys

def install_packages():
    packages = [
        'pandas', 'numpy', 'matplotlib', 'seaborn', 'plotly',
        'sqlalchemy', 'faker', 'sqlite3'
    ]

    for package in packages:
        try:
            __import__(package)
        except ImportError:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Run installation
install_packages()

# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import random
from faker import Faker
import warnings
import sqlite3
from sqlalchemy import create_engine
import time

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('default')

print("✅ All packages installed and imported successfully!")

Installing faker...
✅ All packages installed and imported successfully!


In [3]:
# ==============================================================================
# DATABASE SETUP
# ==============================================================================

def setup_database():
    """Set up SQLite database connection and create tables"""

    # Create database connection
    conn = sqlite3.connect('customer_analytics.db')
    engine = create_engine('sqlite:///customer_analytics.db')

    # Database schema
    schema_sql = """
    -- Drop existing tables if they exist
    DROP TABLE IF EXISTS website_sessions;
    DROP TABLE IF EXISTS customer_interactions;
    DROP TABLE IF EXISTS order_items;
    DROP TABLE IF EXISTS orders;
    DROP TABLE IF EXISTS products;
    DROP TABLE IF EXISTS customers;

    -- Customers table
    CREATE TABLE customers (
        customer_id INTEGER PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        phone TEXT,
        registration_date DATE NOT NULL,
        birth_date DATE,
        gender TEXT,
        city TEXT,
        state TEXT,
        country TEXT,
        acquisition_channel TEXT,
        customer_lifetime_value DECIMAL(10,2),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    -- Products table
    CREATE TABLE products (
        product_id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        category TEXT NOT NULL,
        subcategory TEXT,
        brand TEXT,
        price DECIMAL(10,2) NOT NULL,
        cost DECIMAL(10,2) NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    -- Orders table
    CREATE TABLE orders (
        order_id INTEGER PRIMARY KEY,
        customer_id INTEGER NOT NULL,
        order_date DATE NOT NULL,
        order_status TEXT NOT NULL,
        total_amount DECIMAL(10,2) NOT NULL,
        discount_amount DECIMAL(10,2) DEFAULT 0,
        shipping_cost DECIMAL(10,2) DEFAULT 0,
        payment_method TEXT,
        shipping_address TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );

    -- Order Items table
    CREATE TABLE order_items (
        order_item_id INTEGER PRIMARY KEY,
        order_id INTEGER NOT NULL,
        product_id INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        unit_price DECIMAL(10,2) NOT NULL,
        total_price DECIMAL(10,2) NOT NULL,
        FOREIGN KEY (order_id) REFERENCES orders(order_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id)
    );

    -- Customer Interactions table
    CREATE TABLE customer_interactions (
        interaction_id INTEGER PRIMARY KEY,
        customer_id INTEGER NOT NULL,
        interaction_type TEXT NOT NULL,
        interaction_date DATE NOT NULL,
        channel TEXT,
        duration_minutes INTEGER,
        outcome TEXT,
        notes TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );

    -- Website Sessions table
    CREATE TABLE website_sessions (
        session_id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        session_date DATE NOT NULL,
        session_duration_minutes INTEGER,
        pages_viewed INTEGER,
        device_type TEXT,
        traffic_source TEXT,
        conversion_flag INTEGER DEFAULT 0,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
    """

    # Execute schema creation
    conn.executescript(schema_sql)
    print("✅ Database schema created successfully")

    return conn, engine

# Setup database
conn, engine = setup_database()

✅ Database schema created successfully


In [6]:
# ==============================================================================
# DATA GENERATION
# ==============================================================================

def generate_all_data():
    """Generate all test data for the analytics system"""

    # Initialize Faker
    fake = Faker()
    Faker.seed(42)
    np.random.seed(42)
    random.seed(42)

    print("🔄 Generating test data...")

    # Generate Customers Data
    def generate_customers(n=2000):
        customers = []
        acquisition_channels = ['Google Ads', 'Facebook', 'Email Marketing', 'Referral', 'Direct', 'SEO']

        for i in range(1, n+1):
            customer = {
                'customer_id': i,
                'first_name': fake.first_name(),
                'last_name': fake.last_name(),
                'email': fake.email(),
                'phone': fake.phone_number(),
                'registration_date': fake.date_between(start_date='-2y', end_date='today'),
                'birth_date': fake.date_of_birth(minimum_age=18, maximum_age=80),
                'gender': np.random.choice(['M', 'F', 'Other'], p=[0.45, 0.45, 0.1]),
                'city': fake.city(),
                'state': fake.state(),
                'country': fake.country(),
                'acquisition_channel': np.random.choice(acquisition_channels),
                'customer_lifetime_value': round(np.random.exponential(500), 2)
            }
            customers.append(customer)

        return pd.DataFrame(customers)

    # Generate Products Data
    def generate_products(n=500):
        categories = {
            'Electronics': ['Smartphones', 'Laptops', 'Tablets', 'Accessories'],
            'Clothing': ['Men', 'Women', 'Kids', 'Shoes'],
            'Home & Garden': ['Furniture', 'Appliances', 'Decor', 'Tools'],
            'Books': ['Fiction', 'Non-Fiction', 'Educational', 'Children'],
            'Sports': ['Fitness', 'Outdoor', 'Team Sports', 'Individual Sports']
        }

        brands = ['Apple', 'Samsung', 'Nike', 'Adidas', 'IKEA', 'Sony', 'Dell', 'HP', 'Generic']

        products = []
        for i in range(1, n+1):
            category = np.random.choice(list(categories.keys()))
            subcategory = np.random.choice(categories[category])

            cost = round(np.random.exponential(50) + 10, 2)
            price = round(cost * np.random.uniform(1.5, 3.0), 2)

            product = {
                'product_id': i,
                'product_name': f"{fake.catch_phrase()} {subcategory}",
                'category': category,
                'subcategory': subcategory,
                'brand': np.random.choice(brands),
                'price': price,
                'cost': cost
            }
            products.append(product)

        return pd.DataFrame(products)

    # Generate Orders and Order Items
    def generate_orders_and_items(customers_df, products_df, n_orders=8000):
        orders = []
        order_items = []

        order_statuses = ['Completed', 'Pending', 'Cancelled', 'Returned']
        payment_methods = ['Credit Card', 'Debit Card', 'PayPal', 'Cash', 'Bank Transfer']

        order_id = 1
        order_item_id = 1

        for _ in range(n_orders):
            # Select random customer
            customer_id = np.random.choice(customers_df['customer_id'])
            customer_reg_date = customers_df[customers_df['customer_id'] == customer_id]['registration_date'].iloc[0]

            # Order date should be after registration
            order_date = fake.date_between(start_date=customer_reg_date, end_date='today')

            # Order details
            order = {
                'order_id': order_id,
                'customer_id': customer_id,
                'order_date': order_date,
                'order_status': np.random.choice(order_statuses, p=[0.7, 0.15, 0.1, 0.05]),
                'discount_amount': round(np.random.exponential(10), 2) if np.random.random() < 0.3 else 0,
                'shipping_cost': round(np.random.uniform(5, 25), 2),
                'payment_method': np.random.choice(payment_methods),
                'shipping_address': fake.address().replace('\n', ', ')
            }

            # Generate order items (1-5 items per order)
            num_items = np.random.randint(1, 6)
            order_total = 0

            selected_products = np.random.choice(products_df['product_id'], size=num_items, replace=False)

            for product_id in selected_products:
                product_price = products_df[products_df['product_id'] == product_id]['price'].iloc[0]
                quantity = np.random.randint(1, 4)
                total_price = product_price * quantity
                order_total += total_price

                order_item = {
                    'order_item_id': order_item_id,
                    'order_id': order_id,
                    'product_id': int(product_id),
                    'quantity': quantity,
                    'unit_price': product_price,
                    'total_price': total_price
                }
                order_items.append(order_item)
                order_item_id += 1

            order['total_amount'] = order_total + order['shipping_cost'] - order['discount_amount']
            orders.append(order)
            order_id += 1

        return pd.DataFrame(orders), pd.DataFrame(order_items)

    # Generate Customer Interactions and Website Sessions
    def generate_interactions_and_sessions(customers_df, orders_df):
        interactions = []
        sessions = []

        interaction_types = ['Support Call', 'Email', 'Chat', 'Social Media', 'Review']
        channels = ['Phone', 'Email', 'Website Chat', 'Social Media', 'In-Store']
        outcomes = ['Resolved', 'Escalated', 'Pending', 'Cancelled']

        device_types = ['Desktop', 'Mobile', 'Tablet']
        traffic_sources = ['Direct', 'Google', 'Facebook', 'Email', 'Referral']

        interaction_id = 1
        session_id = 1

        # Generate interactions (some customers have multiple interactions)
        for customer_id in customers_df['customer_id']:
            if np.random.random() < 0.6:  # 60% of customers have interactions
                num_interactions = np.random.poisson(2) + 1
                customer_reg_date = customers_df[customers_df['customer_id'] == customer_id]['registration_date'].iloc[0]

                for _ in range(num_interactions):
                    interaction = {
                        'interaction_id': interaction_id,
                        'customer_id': customer_id,
                        'interaction_type': np.random.choice(interaction_types),
                        'interaction_date': fake.date_between(start_date=customer_reg_date, end_date='today'),
                        'channel': np.random.choice(channels),
                        'duration_minutes': np.random.randint(5, 60),
                        'outcome': np.random.choice(outcomes),
                        'notes': fake.text(max_nb_chars=100)
                    }
                    interactions.append(interaction)
                    interaction_id += 1

        # Generate website sessions
        for customer_id in customers_df['customer_id']:
            customer_reg_date = customers_df[customers_df['customer_id'] == customer_id]['registration_date'].iloc[0]
            customer_orders = orders_df[orders_df['customer_id'] == customer_id]

            # Generate multiple sessions per customer
            num_sessions = np.random.poisson(5) + 1

            for _ in range(num_sessions):
                session_date = fake.date_between(start_date=customer_reg_date, end_date='today')

                # Check if this session resulted in a purchase
                conversion = 1 if len(customer_orders[customer_orders['order_date'] == session_date]) > 0 else 0

                session = {
                    'session_id': session_id,
                    'customer_id': customer_id,
                    'session_date': session_date,
                    'session_duration_minutes': np.random.randint(2, 120),
                    'pages_viewed': np.random.randint(1, 20),
                    'device_type': np.random.choice(device_types),
                    'traffic_source': np.random.choice(traffic_sources),
                    'conversion_flag': conversion
                }
                sessions.append(session)
                session_id += 1

        return pd.DataFrame(interactions), pd.DataFrame(sessions)

    # Generate all data
    customers_df = generate_customers(2000)
    products_df = generate_products(500)
    orders_df, order_items_df = generate_orders_and_items(customers_df, products_df)
    interactions_df, sessions_df = generate_interactions_and_sessions(customers_df, orders_df)

    print(f"✅ Generated:")
    print(f"   - {len(customers_df):,} customers")
    print(f"   - {len(products_df):,} products")
    print(f"   - {len(orders_df):,} orders")
    print(f"   - {len(order_items_df):,} order items")
    print(f"   - {len(interactions_df):,} interactions")
    print(f"   - {len(sessions_df):,} sessions")

    return customers_df, products_df, orders_df, order_items_df, interactions_df, sessions_df

# Generate data
customers_df, products_df, orders_df, order_items_df, interactions_df, sessions_df = generate_all_data()

🔄 Generating test data...
✅ Generated:
   - 2,000 customers
   - 500 products
   - 8,000 orders
   - 24,178 order items
   - 3,572 interactions
   - 12,211 sessions


In [7]:
# ==============================================================================
# LOAD DATA INTO DATABASE
# ==============================================================================

def load_data_to_database():
    """Load all dataframes into the database"""

    print("🔄 Loading data into database...")

    # Load data into tables
    customers_df.to_sql('customers', engine, if_exists='replace', index=False)
    products_df.to_sql('products', engine, if_exists='replace', index=False)
    orders_df.to_sql('orders', engine, if_exists='replace', index=False)
    order_items_df.to_sql('order_items', engine, if_exists='replace', index=False)
    interactions_df.to_sql('customer_interactions', engine, if_exists='replace', index=False)
    sessions_df.to_sql('website_sessions', engine, if_exists='replace', index=False)

    print("✅ All data loaded successfully!")

    # Verify data loading
    tables = ['customers', 'products', 'orders', 'order_items', 'customer_interactions', 'website_sessions']
    print("\n📊 Data Summary:")
    for table in tables:
        count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", engine).iloc[0]['count']
        print(f"   {table}: {count:,} records")

load_data_to_database()

🔄 Loading data into database...
✅ All data loaded successfully!

📊 Data Summary:
   customers: 2,000 records
   products: 500 records
   orders: 8,000 records
   order_items: 24,178 records
   customer_interactions: 3,572 records
   website_sessions: 12,211 records


In [8]:
# ==============================================================================
# CREATE INDEXES FOR PERFORMANCE
# ==============================================================================

def create_indexes():
    """Create database indexes for performance optimization"""

    print("🔄 Creating database indexes...")

    indexes_sql = """
    -- Customer indexes
    CREATE INDEX IF NOT EXISTS idx_customers_registration_date ON customers(registration_date);
    CREATE INDEX IF NOT EXISTS idx_customers_acquisition_channel ON customers(acquisition_channel);

    -- Orders indexes
    CREATE INDEX IF NOT EXISTS idx_orders_customer_id ON orders(customer_id);
    CREATE INDEX IF NOT EXISTS idx_orders_order_date ON orders(order_date);
    CREATE INDEX IF NOT EXISTS idx_orders_status ON orders(order_status);

    -- Order Items indexes
    CREATE INDEX IF NOT EXISTS idx_order_items_order_id ON order_items(order_id);
    CREATE INDEX IF NOT EXISTS idx_order_items_product_id ON order_items(product_id);

    -- Product indexes
    CREATE INDEX IF NOT EXISTS idx_products_category ON products(category);
    CREATE INDEX IF NOT EXISTS idx_products_brand ON products(brand);

    -- Interactions indexes
    CREATE INDEX IF NOT EXISTS idx_interactions_customer_id ON customer_interactions(customer_id);
    CREATE INDEX IF NOT EXISTS idx_interactions_date ON customer_interactions(interaction_date);

    -- Sessions indexes
    CREATE INDEX IF NOT EXISTS idx_sessions_customer_id ON website_sessions(customer_id);
    CREATE INDEX IF NOT EXISTS idx_sessions_date ON website_sessions(session_date);
    """

    conn.executescript(indexes_sql)
    print("✅ Database indexes created successfully!")

create_indexes()

🔄 Creating database indexes...
✅ Database indexes created successfully!


In [9]:
# ==============================================================================
# RFM ANALYSIS AND CUSTOMER SEGMENTATION
# ==============================================================================

def perform_rfm_analysis():
    """Perform comprehensive RFM analysis and customer segmentation"""

    print("🔄 Performing RFM Analysis...")

    rfm_query = """
    WITH customer_metrics AS (
        SELECT
            c.customer_id,
            c.first_name,
            c.last_name,
            c.email,
            c.registration_date,
            c.acquisition_channel,

            -- Recency: Days since last order
            CASE
                WHEN MAX(o.order_date) IS NULL THEN 999
                ELSE julianday('now') - julianday(MAX(o.order_date))
            END as recency_days,

            -- Frequency: Number of orders
            COUNT(DISTINCT o.order_id) as frequency_orders,

            -- Monetary: Total amount spent
            COALESCE(SUM(o.total_amount), 0) as monetary_total

        FROM customers c
        LEFT JOIN orders o ON c.customer_id = o.customer_id
        WHERE o.order_status = 'Completed' OR o.order_status IS NULL
        GROUP BY c.customer_id, c.first_name, c.last_name, c.email,
                 c.registration_date, c.acquisition_channel
    ),

    rfm_scores AS (
        SELECT *,
            -- Recency Score (1-5, where 5 is most recent)
            CASE
                WHEN recency_days <= 30 THEN 5
                WHEN recency_days <= 90 THEN 4
                WHEN recency_days <= 180 THEN 3
                WHEN recency_days <= 365 THEN 2
                ELSE 1
            END as R_score,

            -- Frequency Score (1-5, where 5 is most frequent)
            CASE
                WHEN frequency_orders >= 10 THEN 5
                WHEN frequency_orders >= 7 THEN 4
                WHEN frequency_orders >= 4 THEN 3
                WHEN frequency_orders >= 2 THEN 2
                WHEN frequency_orders >= 1 THEN 1
                ELSE 0
            END as F_score,

            -- Monetary Score (1-5, where 5 is highest value)
            CASE
                WHEN monetary_total >= 1000 THEN 5
                WHEN monetary_total >= 500 THEN 4
                WHEN monetary_total >= 200 THEN 3
                WHEN monetary_total >= 50 THEN 2
                WHEN monetary_total > 0 THEN 1
                ELSE 0
            END as M_score

        FROM customer_metrics
    )

    SELECT *,
        -- Overall RFM Score
        (R_score + F_score + M_score) as RFM_total_score,

        -- Customer Segment based on RFM
        CASE
            WHEN R_score >= 4 AND F_score >= 4 AND M_score >= 4 THEN 'Champions'
            WHEN R_score >= 3 AND F_score >= 3 AND M_score >= 3 THEN 'Loyal Customers'
            WHEN R_score >= 4 AND F_score <= 2 THEN 'New Customers'
            WHEN R_score <= 2 AND F_score >= 3 AND M_score >= 3 THEN 'At Risk'
            WHEN R_score <= 2 AND F_score <= 2 THEN 'Lost Customers'
            WHEN M_score >= 4 THEN 'Big Spenders'
            ELSE 'Potential Loyalists'
        END as customer_segment

    FROM rfm_scores
    ORDER BY RFM_total_score DESC
    """

    rfm_analysis = pd.read_sql(rfm_query, engine)

    print("✅ RFM Analysis completed!")
    print(f"   Total customers analyzed: {len(rfm_analysis):,}")
    print("\n📊 Customer Segment Distribution:")
    segment_dist = rfm_analysis['customer_segment'].value_counts()
    for segment, count in segment_dist.items():
        percentage = (count / len(rfm_analysis)) * 100
        print(f"   {segment}: {count:,} ({percentage:.1f}%)")

    return rfm_analysis

# Perform RFM analysis
rfm_results = perform_rfm_analysis()

🔄 Performing RFM Analysis...
✅ RFM Analysis completed!
   Total customers analyzed: 1,917

📊 Customer Segment Distribution:
   New Customers: 667 (34.8%)
   Loyal Customers: 527 (27.5%)
   Lost Customers: 358 (18.7%)
   Big Spenders: 242 (12.6%)
   At Risk: 49 (2.6%)
   Champions: 40 (2.1%)
   Potential Loyalists: 34 (1.8%)


In [11]:
# ==============================================================================
# COHORT ANALYSIS
# ==============================================================================

def perform_cohort_analysis():
    """Perform customer cohort analysis"""

    print("🔄 Performing Cohort Analysis...")

    cohort_query = """
    WITH customer_cohorts AS (
        SELECT
            customer_id,
            DATE(registration_date, 'start of month') as cohort_month
        FROM customers c
    ),

    first_purchase AS (
        SELECT
            customer_id,
            DATE(MIN(order_date), 'start of month') as first_purchase_month
        FROM orders
        WHERE order_status = 'Completed'
        GROUP BY customer_id
    ),

    cohort_data AS (
        SELECT
            cc.cohort_month,
            DATE(o.order_date, 'start of month') as purchase_month,
            COUNT(DISTINCT cc.customer_id) as customers,
            ROUND((julianday(DATE(o.order_date, 'start of month')) -
                   julianday(cc.cohort_month)) / 30.44, 0) as period_number
        FROM customer_cohorts cc
        JOIN first_purchase fp ON cc.customer_id = fp.customer_id
        JOIN orders o ON cc.customer_id = o.customer_id
        WHERE o.order_status = 'Completed' AND DATE(o.order_date, 'start of month') >= cc.cohort_month
        GROUP BY cc.cohort_month, purchase_month
    ),

    cohort_sizes AS (
        SELECT
            cohort_month,
            COUNT(DISTINCT customer_id) as cohort_size
        FROM customer_cohorts
        GROUP BY cohort_month
    )

    SELECT
        cd.cohort_month,
        cd.period_number,
        cd.customers,
        cs.cohort_size,
        ROUND(100.0 * CAST(cd.customers AS REAL) / cs.cohort_size, 2) as retention_rate
    FROM cohort_data cd
    JOIN cohort_sizes cs ON cd.cohort_month = cs.cohort_month
    WHERE cd.period_number IS NOT NULL
    ORDER BY cd.cohort_month, cd.period_number
    """

    cohort_analysis = pd.read_sql(cohort_query, engine)

    print("✅ Cohort Analysis completed!")
    print(f"   Cohort periods analyzed: {len(cohort_analysis):,}")

    return cohort_analysis

# Perform cohort analysis
cohort_results = perform_cohort_analysis()

🔄 Performing Cohort Analysis...
✅ Cohort Analysis completed!
   Cohort periods analyzed: 322


In [15]:
# ==============================================================================
# REVENUE ANALYSIS
# ==============================================================================

def perform_revenue_analysis():
    """Perform comprehensive revenue analysis"""

    print("🔄 Performing Revenue Analysis...")

    revenue_query = """
    WITH monthly_revenue AS (
        SELECT
            strftime('%Y-%m', order_date) as month_year,
            COUNT(DISTINCT order_id) as total_orders,
            COUNT(DISTINCT customer_id) as unique_customers,
            SUM(total_amount) as monthly_revenue,
            AVG(total_amount) as avg_order_value,

            -- Calculate median order value using percentile
            (
                SELECT total_amount
                FROM orders o2
                WHERE strftime('%Y-%m', o2.order_date) = strftime('%Y-%m', order_date)
                AND o2.order_status = 'Completed'
                ORDER BY total_amount
                LIMIT 1 OFFSET (
                    SELECT COUNT(*) FROM orders o3
                    WHERE strftime('%Y-%m', o3.order_date) = strftime('%Y-%m', order_date)
                    AND o3.order_status = 'Completed'
                ) / 2
            ) as median_order_value

        FROM orders o1
        WHERE order_status = 'Completed'
        GROUP BY strftime('%Y-%m', order_date)
    ),

    revenue_with_growth AS (
        SELECT *,
            -- Calculate month-over-month growth
            LAG(monthly_revenue) OVER (ORDER BY month_year) as prev_month_revenue,
            ROUND(
                100.0 * (monthly_revenue - LAG(monthly_revenue) OVER (ORDER BY month_year)) /
                LAG(monthly_revenue) OVER (ORDER BY month_year), 2
            ) as revenue_growth_percent,

            -- Calculate customer acquisition
            LAG(unique_customers) OVER (ORDER BY month_year) as prev_month_customers,
            unique_customers - LAG(unique_customers) OVER (ORDER BY month_year) as new_customers_acquired,

            -- Rolling 3-month average
            AVG(monthly_revenue) OVER (
                ORDER BY month_year
                ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
            ) as rolling_3month_avg_revenue

        FROM monthly_revenue
    )

    SELECT *
    FROM revenue_with_growth
    ORDER BY month_year
    """

    revenue_analysis = pd.read_sql(revenue_query, engine)

    print("✅ Revenue Analysis completed!")
    print(f"   Months analyzed: {len(revenue_analysis):,}")

    # Display key metrics
    total_revenue = revenue_analysis['monthly_revenue'].sum()
    avg_growth = revenue_analysis['revenue_growth_percent'].mean()

    print(f"   Total Revenue: ${total_revenue:,.2f}")
    print(f"   Average Monthly Growth: {avg_growth:.2f}%")

    return revenue_analysis

# Perform revenue analysis
revenue_results = perform_revenue_analysis()

🔄 Performing Revenue Analysis...
✅ Revenue Analysis completed!
   Months analyzed: 24
   Total Revenue: $4,519,054.56
   Average Monthly Growth: 32.40%


In [51]:
# ==============================================================================
# PRODUCT PERFORMANCE ANALYSIS
# ==============================================================================

def perform_product_analysis():
    """Perform product performance analysis"""

    print("🔄 Performing Product Performance Analysis...")

    product_query = """
    WITH product_performance AS (
        SELECT
            p.product_id,
            p.product_name,
            p.category,
            p.subcategory,
            p.brand,
            p.price,
            p.cost,
            (p.price - p.cost) as profit_per_unit,

            -- Sales Metrics
            COUNT(oi.order_item_id) as times_ordered,
            SUM(oi.quantity) as total_quantity_sold,
            SUM(oi.total_price) as total_revenue,
            SUM(oi.quantity * p.cost) as total_cost,
            SUM(oi.total_price) - SUM(oi.quantity * p.cost) as total_profit,

            -- Calculate profit margin
            ROUND(
                100.0 * (SUM(oi.total_price) - SUM(oi.quantity * p.cost)) /
                SUM(oi.total_price), 2
            ) as profit_margin_percent,

            -- Sales velocity (orders per day)
            ROUND(
                COUNT(oi.order_item_id) /
                CASE
                    WHEN (julianday('now') - julianday(MIN(o.order_date))) = 0 THEN 1
                    ELSE (julianday('now') - julianday(MIN(o.order_date)))
                END, 3
            ) as orders_per_day

        FROM products p
        LEFT JOIN order_items oi ON p.product_id = oi.product_id
        LEFT JOIN orders o ON oi.order_id = o.order_id
        WHERE o.order_status = 'Completed' OR o.order_status IS NULL
        GROUP BY p.product_id, p.product_name, p.category, p.subcategory,
                 p.brand, p.price, p.cost
    ),

    category_rankings AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY category ORDER BY total_revenue DESC) as category_revenue_rank,
            ROW_NUMBER() OVER (ORDER BY total_profit DESC) as overall_profit_rank
        FROM product_performance
    )

    SELECT *
    FROM category_rankings
    WHERE times_ordered > 0  -- Only include products that have been sold
    ORDER BY total_profit DESC
    """

    product_analysis = pd.read_sql(product_query, engine)

    print("✅ Product Analysis completed!")
    print(f"   Products analyzed: {len(product_analysis):,}")

    # Display top performers
    print("\n🏆 Top 10 Most Profitable Products:")
    top_products = product_analysis.head(10)
    display_columns = ['product_name', 'category', 'total_profit', 'profit_margin_percent']
    print(top_products[display_columns].to_string(index=False))

    # Display bottom performers
    print("\n🏆 Bottom 10 Least Profitable Products:")
    bottom_products = product_analysis.tail(10)
    print(bottom_products[display_columns].to_string(index=False))

    return product_analysis

# Perform product analysis
product_results = perform_product_analysis()

🔄 Performing Product Performance Analysis...
✅ Product Analysis completed!
   Products analyzed: 500

🏆 Top 10 Most Profitable Products:
                                      product_name      category  total_profit  profit_margin_percent
        Reverse-engineered 24hour firmware Fiction         Books      58681.80                  66.60
              Integrated composite interface Women      Clothing      36551.06                  62.56
 Multi-layered fresh-thinking capacity Team Sports        Sports      29986.82                  66.25
Quality-focused next generation encoding Furniture Home & Garden      28875.35                  60.47
         Profit-focused tertiary toolset Furniture Home & Garden      27028.27                  63.82
  Monitored content-based portal Individual Sports        Sports      26900.64                  65.72
  Operative disintermediate encryption Educational         Books      25733.79                  43.88
          Devolved zero-defect project Educatio

In [55]:
# ==============================================================================
# HIGH PERFORMANCE PRODUCTS WTH PROFIT MARGIN >=50%
# ==============================================================================

print("🔄 Analyzing ...")

# Query to select high performing products based on profit margin
high_performers_query = """
SELECT
    product_id,
    product_name,
    category,
    subcategory,
    brand,
    price,
    cost,
    total_quantity_sold,
    total_revenue,
    total_profit,
    profit_margin_percent
FROM mv_product_performance
WHERE profit_margin_percent >= 50
ORDER BY profit_margin_percent DESC; -- Order by profit margin for review
"""

high_performers_df = pd.read_sql(high_performers_query, engine)

print(f"✅ Details of {len(high_performers_df)} products with profit margin >= 50%:")
display(high_performers_df)

🔄 Analyzing ...
✅ Details of 326 products with profit margin >= 50%:


,product_id,product_name,category,subcategory,brand,price,cost,total_quantity_sold,total_revenue,total_profit,profit_margin_percent
0,2,Customizable intermediate hub Team Sports,Sports,Team Sports,Nike,107.81,35.96,111,11966.91,7975.35,66.65
1,151,Integrated neutral function Appliances,Home & Garden,Appliances,Nike,76.90,25.66,97,7459.30,4970.28,66.63
2,424,Reverse-engineered 24hour firmware Fiction,Books,Fiction,Sony,978.97,326.95,114,111602.58,74330.28,66.60
3,225,Self-enabling human-resource definition Fiction,Books,Fiction,Nike,196.82,65.91,101,19878.82,13221.91,66.51
4,346,Configurable responsive intranet Men,Clothing,Men,Dell,119.95,40.20,104,12474.80,8294.00,66.49
...,...,...,...,...,...,...,...,...,...,...,...
321,227,Enterprise-wide motivating hub Non-Fiction,Books,Non-Fiction,Samsung,220.43,109.59,113,24908.59,12524.92,50.28
322,252,Implemented object-oriented portal Non-Fiction,Books,Non-Fiction,HP,129.55,64.53,112,14509.60,7282.24,50.19
323,369,Ameliorated 24/7 strategy Smartphones,Electronics,Smartphones,Samsung,152.34,76.00,108,16452.72,8244.72,50.11
324,139,Business-focused context-sensitive solution Ou...,Sports,Outdoor,Sony,266.73,133.16,79,21071.67,10552.03,50.08


In [54]:
# ==============================================================================
# LOW PERFORMANCE PRODUCTS WTH PROFIT MARGIN <50%
# ==============================================================================
print("🔄 Investigating ...")

# Query to select low performing products based on profit margin
low_performers_query = """
SELECT
    product_id,
    product_name,
    category,
    subcategory,
    brand,
    price,
    cost,
    total_quantity_sold,
    total_revenue,
    total_profit,
    profit_margin_percent
FROM mv_product_performance
WHERE profit_margin_percent < 50
ORDER BY profit_margin_percent ASC; -- Order by profit margin for review
"""

low_performers_df = pd.read_sql(low_performers_query, engine)

print(f"✅ Details of {len(low_performers_df)} products with profit margin < 50%:")
display(low_performers_df)

🔄 Investigating ...
✅ Details of 174 products with profit margin < 50%:


,product_id,product_name,category,subcategory,brand,price,cost,total_quantity_sold,total_revenue,total_profit,profit_margin_percent
0,169,Profound well-modulated architecture Children,Books,Children,Generic,41.44,27.61,107,4434.08,1479.81,33.37
1,372,Grass-roots well-modulated attitude Educational,Books,Educational,Sony,160.32,106.36,100,16032.00,5396.00,33.66
2,298,Organic zero tolerance paradigm Laptops,Electronics,Laptops,HP,77.39,51.29,87,6732.93,2270.70,33.73
3,113,Synergized content-based hardware Kids,Clothing,Kids,Generic,29.85,19.76,106,3164.10,1069.54,33.80
4,371,Implemented composite challenge Shoes,Clothing,Shoes,Dell,17.43,11.51,64,1115.52,378.88,33.96
...,...,...,...,...,...,...,...,...,...,...,...
169,83,Multi-lateral scalable synergy Team Sports,Sports,Team Sports,IKEA,111.27,56.10,87,9680.49,4799.79,49.58
170,1,Compatible maximized workforce Individual Sports,Sports,Individual Sports,Generic,83.35,41.96,76,6334.60,3145.64,49.66
171,442,Robust regional methodology Shoes,Clothing,Shoes,Adidas,70.26,35.36,104,7307.04,3629.60,49.67
172,365,Multi-channeled methodical budgetary managemen...,Sports,Team Sports,Generic,21.86,10.98,96,2098.56,1044.48,49.77


In [56]:
# ==============================================================================
# COMPARING HIGH AND LOW PERFORMING PRODUCTS
# ==============================================================================
print("🔄 Comparing ...")

# Calculate summary statistics for low performers
low_performer_summary = low_performers_df[[
    'price', 'cost', 'total_quantity_sold', 'total_revenue', 'total_profit', 'profit_margin_percent'
]].mean().reset_index()
low_performer_summary.columns = ['Metric', 'Low Performer Average']

# Calculate summary statistics for high performers
high_performer_summary = high_performers_df[[
    'price', 'cost', 'total_quantity_sold', 'total_revenue', 'total_profit', 'profit_margin_percent'
]].mean().reset_index()
high_performer_summary.columns = ['Metric', 'High Performer Average']

# Merge summaries for side-by-side comparison
comparison_df = pd.merge(low_performer_summary, high_performer_summary, on='Metric')

print("✅ Comparison of Key Metrics (Averages for Top 10 vs Bottom 10 by Profit):")
display(comparison_df)

print("\nInsights from comparison:")
# Add insights based on the comparison data
insights = []
if comparison_df.iloc[4, 1] < comparison_df.iloc[4, 2]:
    insights.append(f"- High performers have significantly higher average total profit (${comparison_df.iloc[4, 2]:,.2f} vs ${comparison_df.iloc[4, 1]:,.2f}).")
if comparison_df.iloc[2, 1] < comparison_df.iloc[2, 2]:
    insights.append(f"- High performers sell a much larger quantity on average ({comparison_df.iloc[2, 2]:,.0f} vs {comparison_df.iloc[2, 1]:,.0f} units).")
if comparison_df.iloc[3, 1] < comparison_df.iloc[3, 2]:
     insights.append(f"- High performers generate substantially more revenue (${comparison_df.iloc[3, 2]:,.2f} vs ${comparison_df.iloc[3, 1]:,.2f}).")
if comparison_df.iloc[0, 1] < comparison_df.iloc[0, 2]:
    insights.append(f"- High performers tend to have higher average prices (${comparison_df.iloc[0, 2]:,.2f} vs ${comparison_df.iloc[0, 1]:,.2f}).")
if comparison_df.iloc[5, 1] < comparison_df.iloc[5, 2]:
     insights.append(f"- High performers generally have higher average profit margins ({comparison_df.iloc[5, 2]:,.2f}% vs {comparison_df.iloc[5, 1]:,.2f}%).")
else:
     insights.append(f"- Profit margins between high and low performers are relatively similar on average ({comparison_df.iloc[5, 2]:,.2f}% vs {comparison_df.iloc[5, 1]:,.2f}%).")


for insight in insights:
    print(insight)

🔄 Comparing ...
✅ Comparison of Key Metrics (Averages for Top 10 vs Bottom 10 by Profit):


,Metric,Low Performer Average,High Performer Average
0,price,106.055690,140.511534
1,cost,60.559310,56.368436
2,total_quantity_sold,97.189655,96.447853
3,total_revenue,10385.823563,13770.363190
4,total_profit,4456.274943,8257.829816
5,profit_margin_percent,42.245287,59.196258



Insights from comparison:
- High performers have significantly higher average total profit ($8,257.83 vs $4,456.27).
- High performers generate substantially more revenue ($13,770.36 vs $10,385.82).
- High performers tend to have higher average prices ($140.51 vs $106.06).
- High performers generally have higher average profit margins (59.20% vs 42.25%).


In [58]:
# ==============================================================================
# PRODUCT PORTFOLIO ANALYSIS
# ==============================================================================
print("🔄 Performing Analysis...")

# Analyze performance by Category
category_performance = product_results.groupby('category').agg(
    total_category_revenue=('total_revenue', 'sum'),
    total_category_profit=('total_profit', 'sum'),
    total_category_quantity_sold=('total_quantity_sold', 'sum'),
    avg_product_profit_margin=('profit_margin_percent', 'mean'),
    number_of_products=('product_id', 'count')
).sort_values(by='total_category_profit', ascending=False).reset_index()

print("\n📊 Performance by Category:")
display(category_performance)

# Analyze performance by Brand
brand_performance = product_results.groupby('brand').agg(
    total_brand_revenue=('total_revenue', 'sum'),
    total_brand_profit=('total_profit', 'sum'),
    total_brand_quantity_sold=('total_quantity_sold', 'sum'),
    avg_product_profit_margin=('profit_margin_percent', 'mean'),
    number_of_products=('product_id', 'count')
).sort_values(by='total_brand_profit', ascending=False).reset_index()

print("\n📊 Performance by Brand:")
display(brand_performance)

print("✅ Product Portfolio Analysis completed!")

print("\n📊 Visualizing Product Portfolio Analysis...")

# Visualize Performance by Category (Total Profit)
fig_cat_profit = px.bar(category_performance,
                        x='category',
                        y='total_category_profit',
                        title='Product Portfolio Performance by Category (Total Profit)',
                        labels={'total_category_profit': 'Total Profit ($)'},
                        color='category')
fig_cat_profit.update_layout(xaxis_title="Product Category", yaxis_title="Total Profit ($)")
fig_cat_profit.show()

# Visualize Performance by Brand (Total Profit)
fig_brand_profit = px.bar(brand_performance,
                          x='brand',
                          y='total_brand_profit',
                          title='Product Portfolio Performance by Brand (Total Profit)',
                          labels={'total_brand_profit': 'Total Profit ($)'},
                          color='brand')
fig_brand_profit.update_layout(xaxis_title="Brand", yaxis_title="Total Profit ($)")
fig_brand_profit.show()

print("✅ Visualizations created!")

🔄 Performing Analysis...

📊 Performance by Category:


,category,total_category_revenue,total_category_profit,total_category_quantity_sold,avg_product_profit_margin,number_of_products
0,Books,1028805.73,578315.13,7639,53.703929,112
1,Home & Garden,988637.49,550436.68,7545,54.467870,108
2,Sports,844138.28,460230.20,6072,52.614286,91
3,Electronics,833335.41,435935.50,7202,51.279811,106
4,Clothing,755862.01,425915.00,5665,54.550964,83



📊 Performance by Brand:


,brand,total_brand_revenue,total_brand_profit,total_brand_quantity_sold,avg_product_profit_margin,number_of_products
0,Apple,658586.59,350523.58,4600,53.046515,66
1,Sony,573248.74,334037.54,3916,53.954138,58
2,Nike,519551.11,295869.41,3685,54.815091,55
3,HP,543636.34,287967.67,3833,52.325179,56
4,Samsung,497964.22,281106.99,4204,53.069344,61
5,Adidas,445575.57,244775.33,2802,53.848571,42
6,Dell,423726.40,236450.78,3488,54.896200,50
7,Generic,417169.40,227515.06,3911,51.828793,58
8,IKEA,371320.55,192586.15,3684,52.286296,54


✅ Product Portfolio Analysis completed!

📊 Visualizing Product Portfolio Analysis...


✅ Visualizations created!


In [59]:
# ==============================================================================
# PRODUCT PERFOMANCE DASHBOARD
# ==============================================================================
print("🔄 Creating Visualizations ...")

# Ensure necessary dataframes are available (from previous steps)
if 'low_performers_df' not in locals() or 'high_performers_df' not in locals() or 'category_performance' not in locals() or 'brand_performance' not in locals():
    print("⚠️ Required dataframes not found. Please run the previous steps.")
else:
    # 1. Comparison of High vs Low Performers (Bar Chart for Key Metrics)
    fig1 = go.Figure(data=[
        go.Bar(name='Low Performers (Avg)', x=comparison_df['Metric'], y=comparison_df['Low Performer Average']),
        go.Bar(name='High Performers (Avg)', x=comparison_df['Metric'], y=comparison_df['High Performer Average'])
    ])
    fig1.update_layout(title='Average Metrics: Top 10 vs Bottom 10 Profitable Products',
                       xaxis_title="Metric",
                       yaxis_title="Average Value")
    fig1.show()

    # 2. Performance by Category (Bar Chart for Total Profit)
    fig2 = px.bar(category_performance,
                  x='category',
                  y='total_category_profit',
                  title='Total Profit by Product Category',
                  labels={'total_category_profit': 'Total Profit ($)'})
    fig2.update_layout(xaxis_title="Product Category", yaxis_title="Total Profit ($)")
    fig2.show()

    # 3. Performance by Brand (Bar Chart for Total Profit)
    fig3 = px.bar(brand_performance,
                  x='brand',
                  y='total_brand_profit',
                  title='Total Profit by Brand',
                  labels={'total_brand_profit': 'Total Profit ($)'})
    fig3.update_layout(xaxis_title="Brand", yaxis_title="Total Profit ($)")
    fig3.show()

    print("✅ Visualizations created!")

🔄 Creating Visualizations ...


✅ Visualizations created!


In [17]:
# ==============================================================================
# CUSTOMER LIFETIME VALUE CALCULATION
# ==============================================================================

def calculate_customer_lifetime_value():
    """Calculate Customer Lifetime Value (CLV)"""

    print("🔄 Calculating Customer Lifetime Value...")

    clv_query = """
    WITH customer_order_history AS (
        SELECT
            c.customer_id,
            c.registration_date,
            c.acquisition_channel,

            -- Order metrics
            COUNT(o.order_id) as total_orders,
            SUM(o.total_amount) as total_spent,
            AVG(o.total_amount) as avg_order_value,
            MIN(o.order_date) as first_order_date,
            MAX(o.order_date) as last_order_date,

            -- Calculate customer lifespan in days
            CASE
                WHEN COUNT(o.order_id) > 1 THEN
                    julianday(MAX(o.order_date)) - julianday(MIN(o.order_date))
                ELSE 0
            END as lifespan_days,

            -- Days since last order (recency)
            julianday('now') - julianday(MAX(o.order_date)) as days_since_last_order

        FROM customers c
        LEFT JOIN orders o ON c.customer_id = o.customer_id
        WHERE o.order_status = 'Completed' OR o.order_status IS NULL
        GROUP BY c.customer_id, c.registration_date, c.acquisition_channel
    ),

    clv_calculations AS (
        SELECT *,
            -- Purchase frequency (orders per year)
            CASE
                WHEN lifespan_days > 0 THEN
                    ROUND(365.0 * (total_orders - 1) / lifespan_days, 2)
                ELSE 0
            END as purchase_frequency_yearly,

            -- Calculate CLV components
            CASE
                WHEN total_orders > 0 THEN avg_order_value
                ELSE 0
            END as monetary_value,

            -- Estimate customer lifetime (assuming average customer lives 3 years)
            3.0 as estimated_lifetime_years

        FROM customer_order_history
    )

    SELECT *,
        -- Final CLV calculation: AOV × Purchase Frequency × Lifetime
        ROUND(
            monetary_value *
            CASE WHEN purchase_frequency_yearly > 0 THEN purchase_frequency_yearly ELSE 1 END *
            estimated_lifetime_years, 2
        ) as calculated_clv,

        -- Customer value segment
        CASE
            WHEN total_spent >= 1000 THEN 'High Value'
            WHEN total_spent >= 500 THEN 'Medium Value'
            WHEN total_spent >= 100 THEN 'Low Value'
            WHEN total_spent > 0 THEN 'Minimal Value'
            ELSE 'No Purchase'
        END as value_segment

    FROM clv_calculations
    ORDER BY calculated_clv DESC
    """

    clv_analysis = pd.read_sql(clv_query, engine)

    print("✅ Customer Lifetime Value Analysis completed!")
    print(f"   Customers analyzed: {len(clv_analysis):,}")

    # Display value segment distribution
    print("\n💰 Value Segment Distribution:")
    value_dist = clv_analysis['value_segment'].value_counts()
    for segment, count in value_dist.items():
        percentage = (count / len(clv_analysis)) * 100
        print(f"   {segment}: {count:,} ({percentage:.1f}%)")

    return clv_analysis

# Calculate CLV
clv_results = calculate_customer_lifetime_value()

🔄 Calculating Customer Lifetime Value...
✅ Customer Lifetime Value Analysis completed!
   Customers analyzed: 1,917

💰 Value Segment Distribution:
   High Value: 1,482 (77.3%)
   Medium Value: 239 (12.5%)
   Low Value: 153 (8.0%)
   No Purchase: 34 (1.8%)
   Minimal Value: 9 (0.5%)


In [18]:
# ==============================================================================
# CREATE MATERIALIZED VIEWS
# ==============================================================================

def create_materialized_views():
    """Create materialized views for optimized analytics"""

    print("🔄 Creating materialized views...")

    materialized_views_sql = """
    -- Customer Summary View
    DROP TABLE IF EXISTS mv_customer_summary;
    CREATE TABLE mv_customer_summary AS
    SELECT
        c.customer_id,
        c.first_name,
        c.last_name,
        c.email,
        c.registration_date,
        c.acquisition_channel,
        c.city,
        c.state,

        -- Order metrics
        COUNT(o.order_id) as total_orders,
        COALESCE(SUM(o.total_amount), 0) as total_spent,
        COALESCE(AVG(o.total_amount), 0) as avg_order_value,
        MIN(o.order_date) as first_order_date,
        MAX(o.order_date) as last_order_date,

        -- Interaction metrics
        COUNT(ci.interaction_id) as total_interactions,

        -- Session metrics
        COUNT(ws.session_id) as total_sessions,
        AVG(ws.session_duration_minutes) as avg_session_duration,
        SUM(ws.conversion_flag) as total_conversions,

        -- Calculated metrics
        CASE
            WHEN COUNT(o.order_id) > 0 THEN
                julianday('now') - julianday(MAX(o.order_date))
            ELSE 999
        END as days_since_last_order,

        CASE
            WHEN COUNT(ws.session_id) > 0 THEN
                ROUND(100.0 * SUM(ws.conversion_flag) / COUNT(ws.session_id), 2)
            ELSE 0
        END as conversion_rate_percent

    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id AND o.order_status = 'Completed'
    LEFT JOIN customer_interactions ci ON c.customer_id = ci.customer_id
    LEFT JOIN website_sessions ws ON c.customer_id = ws.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name, c.email,
             c.registration_date, c.acquisition_channel, c.city, c.state;

    -- Product Performance View
    DROP TABLE IF EXISTS mv_product_performance;
    CREATE TABLE mv_product_performance AS
    SELECT
        p.product_id,
        p.product_name,
        p.category,
        p.subcategory,
        p.brand,
        p.price,
        p.cost,

        COUNT(oi.order_item_id) as times_ordered,
        SUM(oi.quantity) as total_quantity_sold,
        SUM(oi.total_price) as total_revenue,
        AVG(oi.quantity) as avg_quantity_per_order,

        -- Profitability
        SUM(oi.total_price) - SUM(oi.quantity * p.cost) as total_profit,
        ROUND(
            100.0 * (SUM(oi.total_price) - SUM(oi.quantity * p.cost)) / SUM(oi.total_price), 2
        ) as profit_margin_percent,

        -- Performance ranking
        ROW_NUMBER() OVER (ORDER BY SUM(oi.total_price) DESC) as revenue_rank,
        ROW_NUMBER() OVER (PARTITION BY p.category ORDER BY SUM(oi.total_price) DESC) as category_revenue_rank

    FROM products p
    LEFT JOIN order_items oi ON p.product_id = oi.product_id
    LEFT JOIN orders o ON oi.order_id = o.order_id AND o.order_status = 'Completed'
    GROUP BY p.product_id, p.product_name, p.category, p.subcategory,
             p.brand, p.price, p.cost;
    """

    conn.executescript(materialized_views_sql)
    print("✅ Materialized views created successfully!")

create_materialized_views()

🔄 Creating materialized views...
✅ Materialized views created successfully!


In [19]:
# ==============================================================================
# ETL PIPELINE IMPLEMENTATION
# ==============================================================================

class CustomerAnalyticsETL:
    """ETL Pipeline for Customer Analytics"""

    def __init__(self, db_connection, db_engine):
        self.conn = db_connection
        self.engine = db_engine

    def extract_daily_metrics(self, target_date=None):
        """Extract daily business metrics"""
        if target_date is None:
            target_date = datetime.now().strftime('%Y-%m-%d')

        query = f"""
        SELECT
            '{target_date}' as report_date,
            COUNT(DISTINCT o.order_id) as daily_orders,
            COUNT(DISTINCT o.customer_id) as daily_active_customers,
            SUM(o.total_amount) as daily_revenue,
            AVG(o.total_amount) as daily_avg_order_value,

            -- New customers
            COUNT(DISTINCT CASE WHEN c.registration_date = '{target_date}' THEN c.customer_id END) as new_customers,

            -- Product metrics
            COUNT(DISTINCT oi.product_id) as products_sold,
            SUM(oi.quantity) as total_items_sold,

            -- Channel performance
            COUNT(DISTINCT CASE WHEN ws.traffic_source = 'Google' THEN ws.session_id END) as google_sessions,
            COUNT(DISTINCT CASE WHEN ws.traffic_source = 'Facebook' THEN ws.session_id END) as facebook_sessions,
            COUNT(DISTINCT CASE WHEN ws.traffic_source = 'Direct' THEN ws.session_id END) as direct_sessions,

            -- Conversion metrics
            SUM(ws.conversion_flag) as total_conversions,
            COUNT(ws.session_id) as total_sessions,
            ROUND(100.0 * SUM(ws.conversion_flag) / COUNT(ws.session_id), 2) as conversion_rate

        FROM orders o
        LEFT JOIN customers c ON o.customer_id = c.customer_id
        LEFT JOIN order_items oi ON o.order_id = oi.order_id
        LEFT JOIN website_sessions ws ON o.customer_id = ws.customer_id
            AND ws.session_date = o.order_date
        WHERE o.order_date = '{target_date}' AND o.order_status = 'Completed'
        """

        return pd.read_sql(query, self.engine)

    def transform_customer_segments(self):
        """Transform customer data into segments"""
        # Add risk score calculation
        segments_df = rfm_results.copy()

        segments_df['customer_age_years'] = (
            datetime.now().year - pd.to_datetime(segments_df['registration_date']).dt.year
        )

        segments_df['risk_score'] = segments_df.apply(
            lambda row: self._calculate_risk_score(row), axis=1
        )

        return segments_df

    def _calculate_risk_score(self, row):
        """Calculate customer churn risk score"""
        risk_score = 0

        # Recency risk
        if row['recency_days'] > 180:
            risk_score += 3
        elif row['recency_days'] > 90:
            risk_score += 2
        elif row['recency_days'] > 30:
            risk_score += 1

        # Frequency risk
        if row['frequency_orders'] < 2:
            risk_score += 2

        # Monetary risk
        if row['monetary_total'] < 100:
            risk_score += 1

        return min(risk_score, 5)  # Cap at 5

    def load_dashboard_data(self, transformed_data, table_name):
        """Load transformed data for dashboard"""
        transformed_data.to_sql(
            f'dashboard_{table_name}',
            self.engine,
            if_exists='replace',
            index=False
        )
        print(f"✅ Dashboard data loaded: {table_name}")

# Initialize and run ETL pipeline
def run_etl_pipeline():
    """Execute the complete ETL pipeline"""

    print("🔄 Running ETL Pipeline...")

    etl = CustomerAnalyticsETL(conn, engine)

    # Extract daily metrics for a sample date
    sample_date = '2024-08-15'  # Using a date we know has data
    daily_metrics = etl.extract_daily_metrics(sample_date)

    # Transform customer segments
    customer_segments = etl.transform_customer_segments()

    # Load to dashboard tables
    etl.load_dashboard_data(daily_metrics, 'daily_metrics')
    etl.load_dashboard_data(customer_segments, 'customer_segments')

    print("✅ ETL Pipeline completed successfully!")

    return daily_metrics, customer_segments

# Run ETL pipeline
daily_metrics, customer_segments = run_etl_pipeline()

🔄 Running ETL Pipeline...
✅ Dashboard data loaded: daily_metrics
✅ Dashboard data loaded: customer_segments
✅ ETL Pipeline completed successfully!


In [20]:
# ==============================================================================
# INTERACTIVE DASHBOARD CREATION
# ==============================================================================

def create_comprehensive_dashboard():
    """Create comprehensive analytics dashboard with multiple visualizations"""

    print("🔄 Creating Interactive Dashboard...")

    # Load data for dashboard
    segments_df = customer_segments
    revenue_df = revenue_results
    product_df = product_results

    # Create figure with subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            'Customer Segmentation Distribution',
            'Monthly Revenue Trends',
            'RFM Score Distribution',
            'Top Product Categories by Revenue',
            'Customer Acquisition Channels',
            'Customer Risk Score Distribution'
        ],
        specs=[
            [{"type": "pie"}, {"type": "scatter"}],
            [{"type": "histogram"}, {"type": "bar"}],
            [{"type": "bar"}, {"type": "histogram"}]
        ],
        horizontal_spacing=0.1,
        vertical_spacing=0.1
    )

    # 1. Customer Segmentation Pie Chart
    segment_counts = segments_df['customer_segment'].value_counts()
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#98D8C8']

    fig.add_trace(
        go.Pie(
            labels=segment_counts.index,
            values=segment_counts.values,
            name="Segments",
            marker_colors=colors[:len(segment_counts)]
        ),
        row=1, col=1
    )

    # 2. Monthly Revenue Trends
    fig.add_trace(
        go.Scatter(
            x=revenue_df['month_year'],
            y=revenue_df['monthly_revenue'],
            mode='lines+markers',
            name='Revenue',
            line=dict(color='#1f77b4', width=3),
            marker=dict(size=8)
        ),
        row=1, col=2
    )

    # 3. RFM Score Distribution
    fig.add_trace(
        go.Histogram(
            x=segments_df['RFM_total_score'],
            nbinsx=15,
            name='RFM Scores',
            marker_color='#FF6B6B'
        ),
        row=2, col=1
    )

    # 4. Top Product Categories
    category_revenue = product_df.groupby('category')['total_revenue'].sum().sort_values(ascending=False).head(5)
    fig.add_trace(
        go.Bar(
            x=category_revenue.values,
            y=category_revenue.index,
            orientation='h',
            name='Category Revenue',
            marker_color='#4ECDC4'
        ),
        row=2, col=2
    )

    # 5. Acquisition Channels
    channel_counts = segments_df['acquisition_channel'].value_counts()
    fig.add_trace(
        go.Bar(
            x=channel_counts.index,
            y=channel_counts.values,
            name='Channels',
            marker_color='#45B7D1'
        ),
        row=3, col=1
    )

    # 6. Risk Score Distribution
    fig.add_trace(
        go.Histogram(
            x=segments_df['risk_score'],
            nbinsx=6,
            name='Risk Scores',
            marker_color='#FFEAA7'
        ),
        row=3, col=2
    )

    # Update layout
    fig.update_layout(
        title={
            'text': '🚀 Customer Analytics Dashboard',
            'x': 0.5,
            'font': {'size': 28, 'color': '#2C3E50'}
        },
        height=1400,
        showlegend=False,
        font=dict(size=12),
        plot_bgcolor='white',
        paper_bgcolor='#F8F9FA'
    )

    # Update axes titles
    fig.update_xaxes(title_text="Month", row=1, col=2)
    fig.update_yaxes(title_text="Revenue ($)", row=1, col=2)
    fig.update_xaxes(title_text="RFM Score", row=2, col=1)
    fig.update_yaxes(title_text="Count", row=2, col=1)
    fig.update_xaxes(title_text="Revenue ($)", row=2, col=2)
    fig.update_xaxes(title_text="Channel", row=3, col=1)
    fig.update_yaxes(title_text="Customers", row=3, col=1)
    fig.update_xaxes(title_text="Risk Score", row=3, col=2)
    fig.update_yaxes(title_text="Count", row=3, col=2)

    print("✅ Interactive dashboard created successfully!")
    return fig

# Create dashboard
dashboard = create_comprehensive_dashboard()
dashboard.show()

🔄 Creating Interactive Dashboard...
✅ Interactive dashboard created successfully!


In [22]:
# ==============================================================================
# KPI REPORTING SYSTEM
# ==============================================================================

def generate_comprehensive_kpi_report():
    """Generate comprehensive business KPI report"""

    print("🔄 Generating KPI Report...")

    kpi_queries = {
        'business_overview': """
            SELECT
                COUNT(DISTINCT customer_id) as total_customers,
                COUNT(DISTINCT CASE WHEN total_orders > 0 THEN customer_id END) as active_customers,
                SUM(total_orders) as total_orders,
                ROUND(SUM(total_spent), 2) as total_revenue,
                ROUND(AVG(CASE WHEN total_spent > 0 THEN total_spent END), 2) as avg_customer_value,
                ROUND(AVG(CASE WHEN total_orders > 0 THEN total_orders END), 2) as avg_orders_per_customer
            FROM mv_customer_summary;
        """,

        'customer_metrics': """
            SELECT
                COUNT(CASE WHEN days_since_last_order <= 30 THEN 1 END) as active_last_30_days,
                COUNT(CASE WHEN days_since_last_order > 90 THEN 1 END) as at_risk_customers,
                ROUND(AVG(conversion_rate_percent), 2) as avg_conversion_rate,
                COUNT(CASE WHEN total_interactions > 0 THEN 1 END) as customers_with_interactions
            FROM mv_customer_summary
            WHERE total_orders > 0;
        """,

        'product_metrics': """
            SELECT
                COUNT(DISTINCT product_id) as total_products,
                COUNT(CASE WHEN times_ordered > 0 THEN 1 END) as products_sold,
                ROUND(AVG(CASE WHEN times_ordered > 0 THEN profit_margin_percent END), 2) as avg_profit_margin,
                MAX(total_revenue) as best_product_revenue,
                COUNT(CASE WHEN profit_margin_percent < 20 AND times_ordered > 0 THEN 1 END) as low_margin_products
            FROM mv_product_performance;
        """,

        'segment_performance': """
            SELECT
                customer_segment,
                COUNT(*) as segment_size,
                ROUND(AVG(monetary_total), 2) as avg_segment_value,
                ROUND(AVG(frequency_orders), 2) as avg_segment_frequency,
                COUNT(CASE WHEN risk_score >= 4 THEN 1 END) as high_risk_customers
            FROM dashboard_customer_segments
            GROUP BY customer_segment
            ORDER BY avg_segment_value DESC;
        """
    }

    kpi_report = {}
    for section, query in kpi_queries.items():
        kpi_report[section] = pd.read_sql(query, engine)

    return kpi_report

def display_kpi_dashboard():
    """Display comprehensive KPI dashboard"""

    kpi_data = generate_comprehensive_kpi_report()

    print("=" * 80)
    print("📊 CUSTOMER ANALYTICS SYSTEM - COMPREHENSIVE KPI REPORT")
    print("=" * 80)

    print("\n🏢 BUSINESS OVERVIEW")
    print("-" * 50)
    business = kpi_data['business_overview'].iloc[0]
    active_rate = (business['active_customers'] / business['total_customers']) * 100 if business['total_customers'] > 0 else 0

    print(f"📈 Total Customers: {business['total_customers']:,}")
    print(f"🛒 Active Customers: {business['active_customers']:,} ({active_rate:.1f}%)")
    print(f"📦 Total Orders: {business['total_orders']:,}")
    print(f"💰 Total Revenue: ${business['total_revenue']:,.2f}")
    print(f"💵 Avg Customer Value: ${business['avg_customer_value']:,.2f}")
    print(f"🔄 Avg Orders/Customer: {business['avg_orders_per_customer']:.1f}")

    print("\n👥 CUSTOMER ENGAGEMENT METRICS")
    print("-" * 50)
    customers = kpi_data['customer_metrics'].iloc[0]
    churn_risk_rate = (customers['at_risk_customers'] / business['active_customers']) * 100 if business['active_customers'] > 0 else 0

    print(f"🟢 Active (30 days): {customers['active_last_30_days']:,}")
    print(f"🟡 At Risk (90+ days): {customers['at_risk_customers']:,} ({churn_risk_rate:.1f}%)")
    print(f"📊 Avg Conversion Rate: {customers['avg_conversion_rate']:.2f}%")
    print(f"💬 Customers with Interactions: {customers['customers_with_interactions']:,}")

    print("\n🛍️ PRODUCT PERFORMANCE")
    print("-" * 50)
    products = kpi_data['product_metrics'].iloc[0]
    sell_through_rate = (products['products_sold'] / products['total_products']) * 100 if products['total_products'] > 0 else 0

    print(f"📦 Total Products: {products['total_products']:,}")
    print(f"✅ Products Sold: {products['products_sold']:,} ({sell_through_rate:.1f}%)")
    print(f"💹 Avg Profit Margin: {products['avg_profit_margin']:.2f}%")
    print(f"🏆 Best Product Revenue: ${products['best_product_revenue']:,.2f}")
    print(f"⚠️  Low Margin Products: {products['low_margin_products']:,}")

    print("\n🎯 CUSTOMER SEGMENT ANALYSIS")
    print("-" * 50)
    segments = kpi_data['segment_performance']

    for _, row in segments.iterrows():
        risk_pct = (row['high_risk_customers'] / row['segment_size']) * 100 if row['segment_size'] > 0 else 0
        print(f"🏷️  {row['customer_segment']:<18} | Size: {row['segment_size']:>4,} | "
              f"Value: ${row['avg_segment_value']:>7,.0f} | Risk: {risk_pct:>4.1f}%")

    print("\n" + "=" * 80)

# Generate and display KPI report
display_kpi_dashboard()

🔄 Generating KPI Report...
📊 CUSTOMER ANALYTICS SYSTEM - COMPREHENSIVE KPI REPORT

🏢 BUSINESS OVERVIEW
--------------------------------------------------
📈 Total Customers: 2,000.0
🛒 Active Customers: 1,883.0 (94.2%)
📦 Total Orders: 75,796.0
💰 Total Revenue: $61,240,404.73
💵 Avg Customer Value: $32,522.79
🔄 Avg Orders/Customer: 40.2

👥 CUSTOMER ENGAGEMENT METRICS
--------------------------------------------------
🟢 Active (30 days): 626.0
🟡 At Risk (90+ days): 750.0 (39.8%)
📊 Avg Conversion Rate: 3.17%
💬 Customers with Interactions: 1,123.0

🛍️ PRODUCT PERFORMANCE
--------------------------------------------------
📦 Total Products: 500.0
✅ Products Sold: 500.0 (100.0%)
💹 Avg Profit Margin: 53.30%
🏆 Best Product Revenue: $111,602.58
⚠️  Low Margin Products: 0.0

🎯 CUSTOMER SEGMENT ANALYSIS
--------------------------------------------------
🏷️  Champions          | Size:   40 | Value: $  5,983 | Risk:  0.0%
🏷️  Loyal Customers    | Size:  527 | Value: $  3,729 | Risk:  0.0%
🏷️  At Risk  

In [44]:
# ==============================================================================
# BUSINESS INSIGHTS AND RECOMMENDATIONS
# ==============================================================================

def generate_business_insights():
    """Generate actionable business insights and recommendations"""

    print("🔄 Generating Business Insights...")

    # Analyze customer segments
    segments_df = customer_segments
    product_df = product_results
    revenue_df = revenue_results

    insights = []
    recommendations = []

    # Customer segment insights
    if not segments_df.empty:
      top_segment = segments_df['customer_segment'].value_counts().index[0]
      top_segment_pct = (segments_df['customer_segment'].value_counts().iloc[0] / len(segments_df)) * 100
      insights.append(f"• {top_segment_pct:.1f}% of customers are '{top_segment}' - our largest segment")

    # High-risk customers
    high_risk_count = len(segments_df[segments_df['risk_score'] >= 4])
    high_risk_pct = (high_risk_count / len(segments_df)) * 100 if not segments_df.empty else 0
    insights.append(f"• {high_risk_pct:.1f}% of customers ({high_risk_count:,}) are at high churn risk")

    if high_risk_count > 0:
        recommendations.append({
            'category': '🔴 Customer Retention',
            'priority': 'HIGH',
            'action': f'Launch immediate win-back campaign for {high_risk_count:,} high-risk customers',
            'expected_impact': 'Reduce churn by 20-30%, retain ~${:.0f}K in revenue'.format(
                segments_df[segments_df['risk_score'] >= 4]['monetary_total'].sum() / 1000
            )
        })

    # Revenue trends
    if len(revenue_df) > 1:
        recent_growth = revenue_df['revenue_growth_percent'].dropna().tail(3).mean()
        if recent_growth < 0:
            insights.append(f"• Revenue declining with {recent_growth:.1f}% average growth in recent months")
            recommendations.append({
                'category': '📈 Revenue Recovery',
                'priority': 'HIGH',
                'action': 'Implement aggressive customer acquisition and retention strategies',
                'expected_impact': 'Reverse negative growth trend within 2-3 months'
            })
        else:
            insights.append(f"• Revenue growing at {recent_growth:.1f}% average monthly rate")

    # Product insights (Low Performers)
    if not product_df.empty:
      low_margin_products = product_df[product_df['profit_margin_percent'] < 50]
      if not low_margin_products.empty:
          insights.append(f"• {len(low_margin_products)} products have a profit margin below 50%")
          for idx, row in low_margin_products.iterrows():
              insights.append(f"  - {row['product_name'][:40]}...: {row['profit_margin_percent']:.1f}% profit margin")

      # Product insights (High Performers)
      high_margin_products = product_df[product_df['profit_margin_percent'] > 50]
      if not high_margin_products.empty:
          insights.append(f"• {len(high_margin_products)} products have a profit margin above 50%")
          for idx, row in high_margin_products.iterrows():
              insights.append(f"  - {row['product_name'][:40]}...: {row['profit_margin_percent']:.1f}% profit margin")


    # Acquisition channel insights
    if not segments_df.empty:
      channel_counts = segments_df['acquisition_channel'].value_counts()
      if not channel_counts.empty:
          best_channel = channel_counts.index[0]
          best_channel_count = channel_counts.iloc[0]
          insights.append(f"• {best_channel} is the top acquisition channel with {best_channel_count:,} customers")

          recommendations.append({
              'category': '🎯 Marketing Optimization',
              'priority': 'MEDIUM',
              'action': f'Increase marketing budget allocation to {best_channel} channel',
              'expected_impact': 'Improve customer acquisition efficiency by 25%'
          })


    # New customer opportunity
    if not segments_df.empty:
      new_customers = len(segments_df[segments_df['customer_segment'] == 'New Customers'])
      if new_customers > 0:
          insights.append(f"• {new_customers:,} new customers present immediate upselling opportunities")
          recommendations.append({
              'category': '🆕 Customer Development',
              'priority': 'MEDIUM',
              'action': f'Create onboarding sequence for {new_customers:,} new customers',
              'expected_impact': 'Convert 40% to loyal customers, increase CLV by 50%'
          })


    return insights, recommendations

def display_executive_summary():
    """Display executive summary with insights and recommendations"""

    insights, recommendations = generate_business_insights()

    print("📋 EXECUTIVE SUMMARY")
    print("=" * 60)
    print("🔍 KEY INSIGHTS")
    for insight in insights:
        print(f"  {insight}")

    print(f"\n🎯 STRATEGIC RECOMMENDATIONS")
    print("-" * 60)

    for i, rec in enumerate(recommendations, 1):
        priority_color = "🔴" if rec['priority'] == 'HIGH' else "🟡"
        print(f"\n{i}. {rec['category']} {priority_color}")
        print(f"   📋 Action: {rec['action']}")
        print(f"   📊 Impact: {rec['expected_impact']}")

    print("\n" + "=" * 60)

# Generate executive summary
display_executive_summary()

🔄 Generating Business Insights...
📋 EXECUTIVE SUMMARY
🔍 KEY INSIGHTS
  • 34.8% of customers are 'New Customers' - our largest segment
  • 13.3% of customers (255) are at high churn risk
  • Revenue growing at 11.0% average monthly rate
  • 174 products have a profit margin below 50%
    - Operative disintermediate encryption Edu...: 43.9% profit margin
    - Seamless cohesive contingency Appliances...: 49.5% profit margin
    - Centralized coherent encoding Tablets...: 42.7% profit margin
    - Centralized tertiary hardware Furniture...: 38.0% profit margin
    - Universal systemic projection Kids...: 45.5% profit margin
    - Business-focused object-oriented databas...: 46.2% profit margin
    - Managed user-facing contingency Applianc...: 43.2% profit margin
    - Customer-focused uniform projection Shoe...: 47.6% profit margin
    - Decentralized mobile software Individual...: 48.2% profit margin
    - Multi-lateral reciprocal matrix Smartpho...: 41.0% profit margin
    - Secured sy

In [31]:
# ==============================================================================
# DATA QUALITY VALIDATION
# ==============================================================================

def run_data_quality_checks():
    """Run comprehensive data quality validation"""

    print("🔍 RUNNING DATA QUALITY CHECKS")
    print("=" * 50)

    checks_passed = 0
    total_checks = 0

    # Check 1: Data completeness
    print("\n📊 DATA COMPLETENESS CHECK")
    completeness_query = """
    SELECT
        'customers' as table_name,
        COUNT(*) as total_records,
        SUM(CASE WHEN email IS NULL OR email = '' THEN 1 ELSE 0 END) as missing_emails,
        SUM(CASE WHEN registration_date IS NULL THEN 1 ELSE 0 END) as missing_reg_dates
    FROM customers

    UNION ALL

    SELECT
        'orders' as table_name,
        COUNT(*) as total_records,
        SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) as missing_customer_ids,
        SUM(CASE WHEN total_amount IS NULL OR total_amount <= 0 THEN 1 ELSE 0 END) as invalid_amounts
    FROM orders
    """

    completeness_results = pd.read_sql(completeness_query, engine)
    print(completeness_results.to_string(index=False))

    # Check 2: Data consistency
    print("\n🔗 DATA CONSISTENCY CHECK")
    consistency_query = """
    SELECT
        'orphaned_orders' as check_type,
        COUNT(*) as issue_count
    FROM orders o
    LEFT JOIN customers c ON o.customer_id = c.customer_id
    WHERE c.customer_id IS NULL

    UNION ALL

    SELECT
        'future_orders' as check_type,
        COUNT(*) as issue_count
    FROM orders
    WHERE order_date > date('now')

    UNION ALL

    SELECT
        'negative_amounts' as check_type,
        COUNT(*) as issue_count
    FROM orders
    WHERE total_amount < 0

    UNION ALL

    SELECT
        'duplicate_emails' as check_type,
        COUNT(*) - COUNT(DISTINCT email) as issue_count
    FROM customers
    """

    consistency_results = pd.read_sql(consistency_query, engine)
    print(consistency_results.to_string(index=False))

    # Check 3: Business logic validation
    print("\n💼 BUSINESS LOGIC VALIDATION")
    business_logic_query = """
    SELECT
        'orders_before_registration' as check_type,
        COUNT(*) as issue_count
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    WHERE o.order_date < c.registration_date

    UNION ALL

    SELECT
        'impossible_order_values' as check_type,
        COUNT(*) as issue_count
    FROM orders
    WHERE total_amount > 10000  -- Assuming $10K is max reasonable order

    UNION ALL

    SELECT
        'sessions_before_registration' as check_type,
        COUNT(*) as issue_count
    FROM website_sessions ws
    JOIN customers c ON ws.customer_id = c.customer_id
    WHERE ws.session_date < c.registration_date
    """

    business_logic_results = pd.read_sql(business_logic_query, engine)
    print(business_logic_results.to_string(index=False))

    # Summary
    total_issues = (completeness_results.iloc[:, 2:].sum().sum() +
                   consistency_results['issue_count'].sum() +
                   business_logic_results['issue_count'].sum())

    if total_issues == 0:
        print("\n✅ ALL DATA QUALITY CHECKS PASSED!")
        checks_passed = 3
    else:
        print(f"\n⚠️  Found {total_issues} total data quality issues")
        if total_issues < 10:
            print("   Issues are within acceptable range for test data")
            checks_passed = 2
        else:
            print("   Consider reviewing data generation process")
            checks_passed = 1

    total_checks = 3
    print(f"📊 Data Quality Score: {checks_passed}/{total_checks} ({(checks_passed/total_checks)*100:.0f}%)")

    return checks_passed, total_checks

# Run data quality validation
quality_score = run_data_quality_checks()

🔍 RUNNING DATA QUALITY CHECKS

📊 DATA COMPLETENESS CHECK
table_name  total_records  missing_emails  missing_reg_dates
 customers           2000               0                  0
    orders           8000               0                  0

🔗 DATA CONSISTENCY CHECK
      check_type  issue_count
 orphaned_orders            0
   future_orders            0
negative_amounts            0
duplicate_emails            5

💼 BUSINESS LOGIC VALIDATION
                  check_type  issue_count
  orders_before_registration            0
     impossible_order_values            0
sessions_before_registration            0

⚠️  Found 5 total data quality issues
   Issues are within acceptable range for test data
📊 Data Quality Score: 2/3 (67%)


In [32]:
# ==============================================================================
# PERFORMANCE BENCHMARKING
# ==============================================================================

def benchmark_system_performance():
    """Benchmark query performance and system metrics"""

    print("\n⚡ SYSTEM PERFORMANCE BENCHMARKS")
    print("=" * 50)

    # Test queries for performance
    performance_queries = {
        'Customer Lookup': "SELECT * FROM mv_customer_summary WHERE customer_id = 1000",
        'RFM Analysis Sample': """
            SELECT customer_id, customer_segment, RFM_total_score
            FROM dashboard_customer_segments
            ORDER BY RFM_total_score DESC LIMIT 100
        """,
        'Revenue Aggregation': """
            SELECT strftime('%Y-%m', order_date) as month, SUM(total_amount) as revenue
            FROM orders WHERE order_status = 'Completed'
            GROUP BY strftime('%Y-%m', order_date)
        """,
        'Product Performance': "SELECT * FROM mv_product_performance ORDER BY total_profit DESC LIMIT 50",
        'Complex Join Query': """
            SELECT c.customer_id, c.first_name, COUNT(o.order_id) as orders, SUM(o.total_amount) as total
            FROM customers c
            LEFT JOIN orders o ON c.customer_id = o.customer_id AND o.order_status = 'Completed'
            GROUP BY c.customer_id, c.first_name
            HAVING orders > 5
            ORDER BY total DESC LIMIT 25
        """
    }

    performance_results = {}
    total_time = 0

    for query_name, query in performance_queries.items():
        start_time = time.time()
        result = pd.read_sql(query, engine)
        end_time = time.time()

        execution_time = end_time - start_time
        memory_usage = result.memory_usage(deep=True).sum() / 1024  # KB

        performance_results[query_name] = {
            'time': execution_time,
            'rows': len(result),
            'memory_kb': memory_usage
        }

        total_time += execution_time

        # Performance rating
        if execution_time < 0.1:
            rating = "🟢 Excellent"
        elif execution_time < 0.5:
            rating = "🟡 Good"
        else:
            rating = "🔴 Needs Optimization"

        print(f"\n{query_name}:")
        print(f"  ⏱️  Time: {execution_time:.4f}s | {rating}")
        print(f"  📄 Rows: {len(result):,}")
        print(f"  💾 Memory: {memory_usage:.2f} KB")

    print(f"\n📊 OVERALL PERFORMANCE SUMMARY")
    print(f"   Total Execution Time: {total_time:.4f} seconds")
    print(f"   Average Query Time: {total_time/len(performance_queries):.4f} seconds")

    # Performance grade
    avg_time = total_time / len(performance_queries)
    if avg_time < 0.1:
        grade = "A+ (Production Ready)"
    elif avg_time < 0.3:
        grade = "A (Excellent Performance)"
    elif avg_time < 0.5:
        grade = "B (Good Performance)"
    else:
        grade = "C (Acceptable for Development)"

    print(f"   Performance Grade: {grade}")

    return performance_results

# Run performance benchmarks
perf_results = benchmark_system_performance()


⚡ SYSTEM PERFORMANCE BENCHMARKS

Customer Lookup:
  ⏱️  Time: 0.0061s | 🟢 Excellent
  📄 Rows: 1
  💾 Memory: 0.73 KB

RFM Analysis Sample:
  ⏱️  Time: 0.0023s | 🟢 Excellent
  📄 Rows: 100
  💾 Memory: 7.75 KB

Revenue Aggregation:
  ⏱️  Time: 0.0085s | 🟢 Excellent
  📄 Rows: 24
  💾 Memory: 1.63 KB

Product Performance:
  ⏱️  Time: 0.0069s | 🟢 Excellent
  📄 Rows: 50
  💾 Memory: 17.17 KB

Complex Join Query:
  ⏱️  Time: 0.0184s | 🟢 Excellent
  📄 Rows: 25
  💾 Memory: 2.06 KB

📊 OVERALL PERFORMANCE SUMMARY
   Total Execution Time: 0.0423 seconds
   Average Query Time: 0.0085 seconds
   Performance Grade: A+ (Production Ready)


In [33]:
# ==============================================================================
# FINAL VALIDATION AND SUMMARY
# ==============================================================================

def validate_project_completion():
    """Validate that all project components are working correctly"""

    print("\n🔍 PROJECT COMPLETION VALIDATION")
    print("=" * 60)

    validation_results = {}

    # Check 1: Database tables exist and have data
    required_tables = ['customers', 'products', 'orders', 'order_items',
                      'customer_interactions', 'website_sessions']

    print("📊 Database Validation:")
    tables_valid = True
    for table in required_tables:
        try:
            count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", engine).iloc[0]['count']
            if count > 0:
                print(f"  ✅ {table}: {count:,} records")
            else:
                print(f"  ❌ {table}: No data")
                tables_valid = False
        except Exception as e:
            print(f"  ❌ {table}: Error - {str(e)}")
            tables_valid = False

    validation_results['database'] = tables_valid

    # Check 2: Analytics functions work
    print(f"\n🔬 Analytics Validation:")
    analytics_valid = True

    try:
        # Test RFM analysis
        rfm_test = pd.read_sql("SELECT COUNT(*) as count FROM dashboard_customer_segments", engine)
        if rfm_test.iloc[0]['count'] > 0:
            print("  ✅ RFM Analysis: Working")
        else:
            print("  ❌ RFM Analysis: No results")
            analytics_valid = False
    except Exception as e:
        print(f"  ❌ RFM Analysis: Error - {str(e)}")
        analytics_valid = False

    try:
        # Test materialized views
        mv_test = pd.read_sql("SELECT COUNT(*) as count FROM mv_customer_summary", engine)
        if mv_test.iloc[0]['count'] > 0:
            print("  ✅ Materialized Views: Working")
        else:
            print("  ❌ Materialized Views: No data")
            analytics_valid = False
    except Exception as e:
        print(f"  ❌ Materialized Views: Error - {str(e)}")
        analytics_valid = False

    validation_results['analytics'] = analytics_valid

    # Check 3: Dashboard data exists
    print(f"\n📈 Dashboard Validation:")
    dashboard_valid = True

    try:
        dashboard_test = pd.read_sql("SELECT COUNT(*) as count FROM dashboard_customer_segments", engine)
        if dashboard_test.iloc[0]['count'] > 0:
            print("  ✅ Dashboard Data: Available")
        else:
            print("  ❌ Dashboard Data: Missing")
            dashboard_valid = False
    except Exception as e:
        print(f"  ❌ Dashboard Data: Error - {str(e)}")
        dashboard_valid = False

    validation_results['dashboard'] = dashboard_valid

    # Overall validation
    all_valid = all(validation_results.values())

    if all_valid:
        print(f"\n🎉 PROJECT VALIDATION: ALL SYSTEMS OPERATIONAL! ✅")
        print("   Your Customer Analytics System is ready for portfolio presentation!")
    else:
        print(f"\n⚠️  PROJECT VALIDATION: Some issues detected")
        print("   Please review the errors above before finalizing your project")

    return validation_results

# Run final validation
validation_status = validate_project_completion()

# Saving a final summary to display key metrics one more time
print("\n📊 FINAL SYSTEM METRICS:")
print("-" * 40)
summary_metrics = pd.read_sql("""
SELECT
    (SELECT COUNT(*) FROM customers) as customers,
    (SELECT COUNT(*) FROM orders WHERE order_status = 'Completed') as completed_orders,
    (SELECT COUNT(*) FROM products) as products,
    (SELECT COUNT(DISTINCT customer_segment) FROM dashboard_customer_segments) as segments,
    (SELECT ROUND(SUM(total_amount), 2) FROM orders WHERE order_status = 'Completed') as total_revenue,
    (SELECT COUNT(*) FROM mv_customer_summary) as analytics_records
""", engine)

metrics = summary_metrics.iloc[0]
print(f"👥 Customers Analyzed: {metrics['customers']:,}")
print(f"📦 Orders Processed: {metrics['completed_orders']:,}")
print(f"🛍️  Products Tracked: {metrics['products']:,}")
print(f"🎯 Customer Segments: {metrics['segments']}")
print(f"💰 Total Revenue: ${metrics['total_revenue']:,.2f}")
print(f"📈 Analytics Records: {metrics['analytics_records']:,}")

print(f"\n✨ All Done! ✨")


🔍 PROJECT COMPLETION VALIDATION
📊 Database Validation:
  ✅ customers: 2,000 records
  ✅ products: 500 records
  ✅ orders: 8,000 records
  ✅ order_items: 24,178 records
  ✅ customer_interactions: 3,572 records
  ✅ website_sessions: 12,211 records

🔬 Analytics Validation:
  ✅ RFM Analysis: Working
  ✅ Materialized Views: Working

📈 Dashboard Validation:
  ✅ Dashboard Data: Available

🎉 PROJECT VALIDATION: ALL SYSTEMS OPERATIONAL! ✅
   Your Customer Analytics System is ready for portfolio presentation!

📊 FINAL SYSTEM METRICS:
----------------------------------------
👥 Customers Analyzed: 2,000.0
📦 Orders Processed: 5,626.0
🛍️  Products Tracked: 500.0
🎯 Customer Segments: 7.0
💰 Total Revenue: $4,519,054.56
📈 Analytics Records: 2,000.0

✨ All Done! ✨
